In [63]:
import numpy as np
import sympy as sp
from scipy.optimize import minimize
from scipy.special import roots_legendre
import scipy.linalg as la
import matplotlib.pyplot as plt
from FEM import *
from tools import *
from shape_functions import *
# plt.style.use('default')
import copy
fontsize = 15

In [64]:
num_elems = 4
domain = (0, 1)
p = 3
mesh = np.linspace(domain[0], domain[1], num_elems+1)
a = .5*1
xb = 0.8
if a == 50:
    U_init = 1.585854059271320
elif a == 0.5:
    U_init = 0.03559183822564316
exact_func = exact_fn(a = a, xb=xb)
rhs_func = rhs_fn(a=a, xb=xb)
BCs = (exact_func(domain[0]), exact_func(domain[-1]))


In [83]:


def FEM_1D(shape_class = Hierarchical, p = 3, num_elems = 3, domain = (0, 1),rhs_func = rhs_fn(a=50, xb=0.8), exact_func=exact_fn(0.5,0.8), BCs = (0, 0), verbose = False):
    N=6
    mesh = np.linspace(domain[0], domain[1], num_elems+1)
    ori_phi_phip = {'phis': [], 'phips': []}
    for elem in range(num_elems):
        scale = [mesh[elem], mesh[elem+1]]
        phis, phips = shape_class(scale, p)
        ori_phi_phip['phis'].append(phis)
        ori_phi_phip['phips'].append(phips)


    linear_phi_phip = {'phis': [], 'phips': []}  # Linear
    for elem in range(num_elems):
        linear_phis = []
        linear_phips = []
        for idx in range(len(ori_phi_phip['phis'][elem])):
            if ori_phi_phip['phis'][elem][idx].p < 2:
                phi = ori_phi_phip['phis'][elem][idx]
                phip = ori_phi_phip['phips'][elem][idx]
                linear_phi_phip['phis'].append(phi)
                linear_phi_phip['phips'].append(phip)
                linear_phis.append(phi)
                linear_phips.append(phip)
        linear_K_sub = np.zeros((len(linear_phips), len(linear_phips)))
        for indx, x in np.ndenumerate(linear_K_sub):
            linear_K_sub[indx] = G_integrate(
                mul(linear_phips[indx[0]], linear_phips[indx[-1]]), N=6, scale=linear_phips[indx[0]].scale)
            if abs(linear_K_sub[indx]) < 1e-10:
                linear_K_sub[indx] = 0
        # print('K_sub', K_sub)
        linear_F_sub = np.zeros(len(linear_K_sub))
        for indx in range(len(linear_F_sub)):
            linear_F_sub[indx] = G_integrate(
                mul(rhs_func, linear_phis[indx]), N=N, scale=linear_phis[indx].scale)
            # print(phis[indx](mesh[i]))
        if elem == 0:
            K = linear_K_sub
            F = linear_F_sub
        else:
            K = assemble(K, linear_K_sub)
            F = assemble(F, linear_F_sub)
            
    linear_num = len(F)

    nonlinear_phi_phip = {'phis': [], 'phips': []}
    for order in range(2, p+1):  # Non Linear
        # print('order', order)
        for elem in range(num_elems):
            for idx in range(len(ori_phi_phip['phis'][elem])):
                if (ori_phi_phip['phis'][elem][idx].p == order) or (ori_phi_phip['phips'][elem][idx].p == order):
                    nonlinear_phi = ori_phi_phip['phis'][elem][idx]
                    nonlinear_phip = ori_phi_phip['phips'][elem][idx]
                    nonlinear_phi_phip['phis'].append(nonlinear_phi)
                    nonlinear_phi_phip['phips'].append(nonlinear_phip)
                    nonlinear_K_sub = np.zeros((2, 2))
                    # print('nonlinear_phip', nonlinear_phip.p)
                    # print(G_integrate(mul(nonlinear_phip, nonlinear_phip),N=N, scale=nonlinear_phip.scale))
                    
                    nonlinear_K_sub[-1, -1] = G_integrate(mul(nonlinear_phip, nonlinear_phip),N=N, scale=nonlinear_phip.scale)
                    nonlinear_F_sub = np.zeros(2)
                    nonlinear_F_sub[-1] = G_integrate(mul(rhs_func, nonlinear_phi), N=N, scale=nonlinear_phi.scale)

                    K = assemble(K, nonlinear_K_sub)
                    F = assemble(F, nonlinear_F_sub)
                else:
                    pass
                
    # Applying boundary condition
    
    K[0, 1:] = 0.0 
    K[linear_num-1, :linear_num-1] = 0.0
    F[0] = BCs[0]* K[0, 0] # -= or = ??
    F[linear_num-1] = BCs[-1] * K[linear_num-1, linear_num-1]

    U = -la.solve(K, F)
    # print(F)
    phi_phip = {'phis': [], 'phips': []}
    phi_phip['phis'] = joint_funcs(linear_phi_phip['phis']) + nonlinear_phi_phip['phis']
    phi_phip['phips'] = joint_funcs(linear_phi_phip['phips']) + nonlinear_phi_phip['phips']
    u_list = []
    print(len(phi_phip['phis']))
    print(len(U))
    for i in range(len(phi_phip['phis'])):
        u_list.append(mul(U[i], phi_phip['phis'][i]))
    uh = plus(u_list)
    eigenvalues = np.linalg.eigvals(K)
    cont_K = max(eigenvalues)/min(eigenvalues)
    if verbose == True:
        print(f"Shape class: {shape_class.__name__}, Number of elements: {num_elems}, Polynomial order:{p},  Domain: {domain}, Boundary conditions: {BCs}")
        x_data = np.linspace(domain[0], domain[1], 101)
        plt.plot(x_data, exact_func(x_data), label='Analytical solution')
        plt.plot(x_data, uh(x_data), label='FEM solution {} elements'.format(num_elems))
        for i in range(len(phi_phip['phis'])):
            func = phi_phip['phis'][i]
            plt.plot(x_data, U[i]*func(x_data))
        plt.legend()
        plt.show()
    return U, phi_phip, uh, cont_K, K



In [73]:

p_list = [3]
for p in range(1, 6):

    U_p, phi_phip_p, uh_p, cont_K_p, K = FEM_1D(shape_class = Hierarchical,p = p, num_elems = 5, domain = domain,rhs_func = rhs_func,exact_func=exact_func, BCs = BCs, verbose = False)
    print(cont_K_p)

9.472135954999576
9.472135954999576
9.472135954999576
9.472135954999576
9.472135954999576


In [67]:
K = [[ 2., 0,  0.,  0.,  0.,  0.,  0.],
 [-2,  4., -2.,  0.,  0.,  0.,  0.],
 [ 0., -0,  2.,  0.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  4.,  0.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  4.,  0.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  4.,  0.],
 [ 0.,  0.,  0.,  0.,  0.,  0.,  4.]]
eigenvalues = np.linalg.eigvals(K)
eigenvalues

array([4., 2., 2., 4., 4., 4., 4.])

In [74]:

p_list = [5]
for p in range(1, 6):

    U_q, phi_phip_q, uh_q, cont_K_q, K_q = FEM_1D(shape_class = linear,p = p, num_elems = p, domain = domain,rhs_func = rhs_func,exact_func=exact_func, BCs = BCs, verbose = False)
    print(cont_K_q)

1.0
2.0


IndexError: index 4 is out of bounds for axis 0 with size 4

In [89]:
U_q, phi_phip_q, uh_q, cont_K_q, K_q = FEM_1D(shape_class = linear,p = 11, num_elems = 31, domain = domain,rhs_func = rhs_func,exact_func=exact_func, BCs = BCs, verbose = False)
print(cont_K_q)

32
32
388.8121344932208


In [69]:
eigenvalues = np.linalg.eigvals(K_q)
eigenvalues

array([ 0.97886967,  3.81966011,  8.24429495, 13.81966011, 20.        ,
       39.02113033, 36.18033989, 26.18033989, 31.75570505, 10.        ,
       10.        ])

In [93]:
functions = np.zeros(6)
if len(functions)%3==0:
        chunk_size = 3
elif len(functions)%2==0:
        chunk_size = 2
        
print(chunk_size)

3


In [95]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial import Delaunay
from scipy.sparse import lil_matrix
from scipy.sparse.linalg import spsolve

# 基本设置
hmax = 0.25
coef = lambda x, y: 1
righthand = lambda x, y: 2

# 初始化网格
p = np.mgrid[0:1:hmax, 0:1:hmax].reshape(2,-1)
tri = Delaunay(p.T)

for i in range(5):
    # 细化网格
    p = refine_mesh(p, tri.simplices)  # 自定义函数，需要实现
    tri = Delaunay(p.T)
    plt.triplot(p[0], p[1], tri.simplices)
    
    # 处理边界条件
    boundary_nodes = np.unique(tri.convex_hull)
    A = assemble_matrix_2D2(coef, p, tri.simplices)  # 自定义函数，需要实现
    b = assemble_vector_2D(righthand, p, tri.simplices)  # 自定义函数，需要实现
    A, b = treat_boundary_condition(A, b, p, boundary_nodes)  # 自定义函数，需要实现
    
    # 解线性方程组
    u = spsolve(A, b)
    
    # 绘图
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    ax.plot_trisurf(p[0], p[1], u, cmap=plt.cm.jet, linewidth=0.2)
    plt.show()


NameError: name 'refine_mesh' is not defined

In [1]:
import numpy as np

def compute_affine_transformation(src, dst):
    """Compute the affine transformation matrix from src to dst."""
    # Construct the matrix A
    A = np.array([
        [src[0][0], src[0][1], 1, 0, 0, 0],
        [0, 0, 0, src[0][0], src[0][1], 1],
        [src[1][0], src[1][1], 1, 0, 0, 0],
        [0, 0, 0, src[1][0], src[1][1], 1],
        [src[2][0], src[2][1], 1, 0, 0, 0],
        [0, 0, 0, src[2][0], src[2][1], 1]
    ])
    # Construct the matrix b
    b = np.array([
        dst[0][0],
        dst[0][1],
        dst[1][0],
        dst[1][1],
        dst[2][0],
        dst[2][1]
    ])
    
    # Solve the system
    x = np.linalg.solve(A, b)
    
    # Construct the transformation matrix
    transform = np.array([
        [x[0], x[1], x[2]],
        [x[3], x[4], x[5]],
        [0, 0, 1]
    ])
    return transform

src_tri = np.array([[0,0], [1,2], [2,1]])
dst_tri = np.array([[0, 0], [1, 0], [0, 1]])

transformation = compute_affine_transformation(src_tri, dst_tri)
jacobian = transformation[:2, :2]
print(jacobian)


[[-0.33333333  0.66666667]
 [ 0.66666667 -0.33333333]]
